In [1]:
import os
import sys
import numpy as np
import pydot
import rmgpy.tools.fluxdiagram
import rmgpy.chemkin

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('/home/moon/autoscience/reaction_calculator/database')
import database_fun

## Settings

In [2]:
# Options controlling the individual flux diagram renderings:
program = 'dot'  # The program to use to lay out the nodes and edges
max_node_count = 35  # The maximum number of nodes to show in the diagram
max_edge_count = 35  # The maximum number of edges to show in the diagram
concentration_tol = 1e-6  # The lowest fractional concentration to show (values below this will appear as zero)
species_rate_tol = 1e-6  # The lowest fractional species rate to show (values below this will appear as zero)
max_node_pen_width = 7.0  # The thickness of the border around a node at maximum concentration
max_edge_pen_width = 9.0  # The thickness of the edge at maximum species rate
radius = 1  # The graph radius to plot around a central species
central_reaction_count = None  # The maximum number of reactions to draw from each central species (None draws all)
# If radius > 1, then this is the number of reactions from every species

# Options controlling the ODE simulations:
initial_time = 1e-12  # The time at which to initiate the simulation, in seconds
time_step = 10 ** 0.1  # The multiplicative factor to use between consecutive time points
abs_tol = 1e-16  # The absolute tolerance to use in the ODE simluations
rel_tol = 1e-8  # The relative tolerance to use in the ODE simulations

# Options controlling the generated movie:
video_fps = 6  # The number of frames per second in the generated movie
initial_padding = 5  # The number of seconds to display the initial fluxes at the start of the video
final_padding = 5  # The number of seconds to display the final fluxes at the end of the video




## Load Mechanisms

In [3]:
# species_path = None
java = False            # always False
settings = None
chemkin_output = ''     # this will be generated automatically
central_species_list = None
superimpose = False     # this will always be false, delete it
save_states = False
read_states = False     # fine to keep this always false and delete relevant code below
diffusion_limited = True
check_duplicates = True

rmg_input_file = '/home/moon/autoscience/fuels/simplified_input.py'  # for conditions T = 830, P = 10 atm

diagram_base_name = 'three_way_compare'  # is this the one or is it the other?? Aramco should be slowest, so I think it's this
os.makedirs(diagram_base_name, exist_ok=True)
mech_1_inp = '/home/moon/autoscience/aramco/flux_diagram/chem_annotated.inp'
mech_1_dict = '/home/moon/autoscience/aramco/species_dictionary.txt'

mech_1_label = 'AramcoMech3.0'
t1_ignition = 0.06535855787324196  # the ignition time for model 1
mech_2_inp = '/home/moon/autoscience/fuels/butane_20240126/chem_annotated.inp'
mech_2_dict = '/home/moon/autoscience/fuels/butane_20240126/species_dictionary.txt'
mech_2_label = 'RMG-min-1'
t2_ignition = 0.006677756747669057
mech_3_inp = '/home/moon/autoscience/fuels/butane_20240501/chem_annotated.inp'
mech_3_dict = '/home/moon/autoscience/fuels/butane_20240501/species_dictionary.txt'
mech_3_label = 'RMG-min-7'
t3_ignition = 0.02934108246263338

generate_images = False
species_path = '/home/moon/autoscience/fuels/species'
print('Loading RMG job 1...')
rmg_job1 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_1_inp,
    mech_1_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates
)

print('Loading RMG job 2...')
rmg_job2 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_2_inp,
    mech_2_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates
)

print('Loading RMG job 3...')
rmg_job3 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_3_inp,
    mech_3_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates
)


Loading RMG job 1...


Loading RMG job 2...


Loading RMG job 3...


## Simulation

In [4]:
print('Conducting simulation of reaction 1')
times1, concentrations1, reaction_rates1 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job1.reaction_model,
    rmg_job1.reaction_systems[0],
    settings
)

print('Conducting simulation of reaction 2')
times2, concentrations2, reaction_rates2 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job2.reaction_model,
    rmg_job2.reaction_systems[0],
    settings
)

print('Conducting simulation of reaction 3')
times3, concentrations3, reaction_rates3 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job3.reaction_model,
    rmg_job3.reaction_systems[0],
    settings
)

Conducting simulation of reaction 1
Conducting simulation of reaction 2
Conducting simulation of reaction 3


# Compute/assemble species concentrations and fluxes into convenient form

In [5]:
# Get the RMG species and reactions objects
species_list1 = rmg_job1.reaction_model.core.species[:]
reaction_list1 = rmg_job1.reaction_model.core.reactions[:]
num_species1 = len(species_list1)

species_list2 = rmg_job2.reaction_model.core.species[:]
reaction_list2 = rmg_job2.reaction_model.core.reactions[:]
num_species2 = len(species_list2)

species_list3 = rmg_job3.reaction_model.core.species[:]
reaction_list3 = rmg_job3.reaction_model.core.reactions[:]
num_species3 = len(species_list3)

In [6]:
# Compute the rates between each pair of species (build up big matrices)
species_rates1 = np.zeros((len(times1), num_species1, num_species1), float)
for index1, reaction1 in enumerate(reaction_list1):
    rate1 = reaction_rates1[:, index1]
    if not reaction1.pairs: reaction1.generate_pairs()
    for reactant1, product1 in reaction1.pairs:
        reactant_index1 = species_list1.index(reactant1)
        product_index1 = species_list1.index(product1)
        species_rates1[:, reactant_index1, product_index1] += rate1
        species_rates1[:, product_index1, reactant_index1] -= rate1
        
species_rates2 = np.zeros((len(times2), num_species2, num_species2), float)
for index2, reaction2 in enumerate(reaction_list2):
    rate2 = reaction_rates2[:, index2]
    if not reaction2.pairs: reaction2.generate_pairs()
    for reactant2, product2 in reaction2.pairs:
        reactant_index2 = species_list2.index(reactant2)
        product_index2 = species_list2.index(product2)
        species_rates2[:, reactant_index2, product_index2] += rate2
        species_rates2[:, product_index2, reactant_index2] -= rate2
        
species_rates3 = np.zeros((len(times3), num_species3, num_species3), float)
for index3, reaction3 in enumerate(reaction_list3):
    rate3 = reaction_rates3[:, index3]
    if not reaction3.pairs: reaction3.generate_pairs()
    for reactant3, product3 in reaction3.pairs:
        reactant_index3 = species_list3.index(reactant3)
        product_index3 = species_list3.index(product3)
        species_rates3[:, reactant_index3, product_index3] += rate3
        species_rates3[:, product_index3, reactant_index3] -= rate3

In [7]:
# Determine the maximum concentration for each species and the maximum overall concentration
max_concentrations1 = np.max(np.abs(concentrations1), axis=0)
max_concentration1 = np.max(max_concentrations1)

# Determine the maximum reaction rates
max_reaction_rates1 = np.max(np.abs(reaction_rates1), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates1 = np.max(np.abs(species_rates1), axis=0)
max_species_rate1 = np.max(max_species_rates1)
species_index1 = max_species_rates1.reshape((num_species1 * num_species1)).argsort()


max_concentrations2 = np.max(np.abs(concentrations2), axis=0)
max_concentration2 = np.max(max_concentrations2)

# Determine the maximum reaction rates
max_reaction_rates2 = np.max(np.abs(reaction_rates2), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates2 = np.max(np.abs(species_rates2), axis=0)
max_species_rate2 = np.max(max_species_rates2)
species_index2 = max_species_rates2.reshape((num_species2 * num_species2)).argsort()


max_concentrations3 = np.max(np.abs(concentrations3), axis=0)
max_concentration3 = np.max(max_concentrations3)

# Determine the maximum reaction rates
max_reaction_rates3 = np.max(np.abs(reaction_rates3), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates3 = np.max(np.abs(species_rates3), axis=0)
max_species_rate3 = np.max(max_species_rates3)
species_index3 = max_species_rates3.reshape((num_species3 * num_species3)).argsort()



max_species_rate_total = max(max_species_rate1, max_species_rate2, max_species_rate3)
max_concentration_total = max(max_concentration1, max_concentration2, max_concentration3)

# Determine nodes and edges to include for each model

In [8]:
nodes1 = []
edges1 = []
for i in range(num_species1 * num_species1):
    product_index1, reactant_index1 = divmod(species_index1[-i - 1], num_species1)
    if reactant_index1 > product_index1:
        # Both reactant -> product and product -> reactant are in this list, so only keep one of them
        continue
    if max_species_rates1[reactant_index1, product_index1] == 0:
        break
    if reactant_index1 not in nodes1 and len(nodes1) < max_node_count: nodes1.append(reactant_index1)
    if product_index1 not in nodes1 and len(nodes1) < max_node_count: nodes1.append(product_index1)
    if [reactant_index1, product_index1] not in edges1 and [product_index1, reactant_index1] not in edges1:
        edges1.append([reactant_index1, product_index1])
    if len(nodes1) > max_node_count:
        break
    if len(edges1) >= max_edge_count:
        break
        
nodes2 = []
edges2 = []
for i in range(num_species2 * num_species2):
    product_index2, reactant_index2 = divmod(species_index2[-i - 1], num_species2)
    if reactant_index2 > product_index2:
        # Both reactant -> product and product -> reactant are in this list, so only keep one of them
        continue
    if max_species_rates2[reactant_index2, product_index2] == 0:
        break
    if reactant_index2 not in nodes2 and len(nodes2) < max_node_count: nodes2.append(reactant_index2)
    if product_index2 not in nodes2 and len(nodes2) < max_node_count: nodes2.append(product_index2)
    if [reactant_index2, product_index2] not in edges2 and [product_index2, reactant_index2] not in edges2:
        edges2.append([reactant_index2, product_index2])
    if len(nodes2) > max_node_count:
        break
    if len(edges2) >= max_edge_count:
        break
  
        
nodes3 = []
edges3 = []
for i in range(num_species3 * num_species3):
    product_index3, reactant_index3 = divmod(species_index3[-i - 1], num_species3)
    if reactant_index3 > product_index3:
        # Both reactant -> product and product -> reactant are in this list, so only keep one of them
        continue
    if max_species_rates3[reactant_index3, product_index3] == 0:
        break
    if reactant_index3 not in nodes3 and len(nodes3) < max_node_count: nodes3.append(reactant_index3)
    if product_index3 not in nodes3 and len(nodes3) < max_node_count: nodes3.append(product_index3)
    if [reactant_index3, product_index3] not in edges3 and [product_index3, reactant_index3] not in edges3:
        edges3.append([reactant_index3, product_index3])
    if len(nodes3) > max_node_count:
        break
    if len(edges3) >= max_edge_count:
        break


# Create mapping of species indices between the three models

In [9]:
species2_to_1 = {}
species2_to_1 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list2)):
    for j in range(len(species_list1)):
        if species_list2[i].is_isomorphic(species_list1[j]):
            species2_to_1[i] = j
            break
    else:
        species2_to_1[i] = -1

species1_to_2 = {}
species1_to_2 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list1)):
    for j in range(len(species_list2)):
        if species_list1[i].is_isomorphic(species_list2[j]):
            species1_to_2[i] = j
            break
    else:
        species1_to_2[i] = -1
        
        
species3_to_1 = {}
species3_to_1 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list3)):
    for j in range(len(species_list1)):
        if species_list3[i].is_isomorphic(species_list1[j]):
            species3_to_1[i] = j
            break
    else:
        species3_to_1[i] = -1
        
species1_to_3 = {}
species1_to_3 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list1)):
    for j in range(len(species_list3)):
        if species_list1[i].is_isomorphic(species_list3[j]):
            species1_to_3[i] = j
            break
    else:
        species1_to_3[i] = -1
        
        
species3_to_2 = {}
species3_to_2 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list3)):
    for j in range(len(species_list2)):
        if species_list3[i].is_isomorphic(species_list2[j]):
            species3_to_2[i] = j
            break
    else:
        species3_to_2[i] = -1
        
species2_to_3 = {}
species2_to_3 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list2)):
    for j in range(len(species_list3)):
        if species_list2[i].is_isomorphic(species_list3[j]):
            species2_to_3[i] = j
            break
    else:
        species2_to_3[i] = -1

# Specify important reactions to label on graph

In [10]:
# Note that labels were added manually after the fact because it led to a prettier layout

indices_to_label = [
    4732, 4752, 324, 281, 263, 4778, 313, 278  # uses database indices
]
ref_rxns = [database_fun.index2reaction(i) for i in indices_to_label]

def label_flux_pair(r, p):
    # return true if it's in the list of reactions to label
    for ref_rxn in ref_rxns:
        for reactant in ref_rxn.reactants:
            if r.is_isomorphic(reactant):
                for product in ref_rxn.products:
                    if p.is_isomorphic(product):
                        return indices_to_label[ref_rxns.index(ref_rxn)]

        for product in ref_rxn.products:
            if r.is_isomorphic(product):
                for reactant in ref_rxn.reactants:
                    if p.is_isomorphic(reactant):
                        return indices_to_label[ref_rxns.index(ref_rxn)]


In [11]:
# Function to grab and generate the image for a given species
def get_image_path(species):
    species_index = str(species) + '.png'
    image_path = ''
    if not species_path or not os.path.exists(species_path):  # species_path is defined while loading the mechanism
        raise OSError
    for root, dirs, files in os.walk(species_path):
        for f in files:
            if f == species_index:
                image_path = os.path.join(root, f)
                break
    if not image_path:
        image_path = os.path.join(species_path, species_index)
        species.molecule[0].draw(image_path)
    return image_path

# Build the Graph

In [12]:
# Create the graph
rmg1_color = matplotlib.colors.to_hex((1.0, 0.92, 0.0))
rmg7_color = matplotlib.colors.to_hex(matplotlib.colors.CSS4_COLORS['forestgreen'])
aramco_color = matplotlib.colors.to_hex((0.18627451, 0.48823529, 0.94117647))

# add alpha specified by adding hex digits before RGB values
alpha = 0.9
alpha_hex = hex(int(alpha * 255))[2:]
aramco_color = f'#{aramco_color[1:]}{alpha_hex}'
rmg1_color = f'#{rmg1_color[1:]}{alpha_hex}'
rmg7_color = f'#{rmg7_color[1:]}{alpha_hex}'

# Grab the fluxes from the time closest (without going over) to official ignition delay time
t1 = np.abs(times1 - t1_ignition).argmin()
if times1[t1] > t1_ignition:
    t1 -= 1
t2 = np.abs(times2 - t2_ignition).argmin()
if times2[t2] > t2_ignition:
    t2 -= 1
t3 = np.abs(times3 - t3_ignition).argmin()
if times3[t3] > t3_ignition:
    t3 -= 1


label_automatically = False  # Turned off because manual labels upset the graph placements less
assert -1 not in nodes1
assert -1 not in nodes2
assert -1 not in nodes3


slope = -max_node_pen_width / np.log10(concentration_tol)
graph = pydot.Dot('flux_diagram', graph_type='digraph', overlap="false")
graph.set_fontname('sans')
graph.set_fontsize('10')

# ----------------------------ADD NODES ------------------------------#
# For Mechanism 1
for index1 in nodes1:
    nodewidths = np.zeros(3)  # keep track of species concentrations/nodewidths for all 3 mechanisms
    species1 = species_list1[index1]
    node1 = pydot.Node(name=str(species1))
    concentration1 = concentrations1[t1, index1] / max_concentration_total
    if concentration1 < concentration_tol:
        penwidth = 0.0
    else:
        penwidth = round(slope * np.log10(concentration1) + max_node_pen_width, 3)
        nodewidths[0] = penwidth
    node1.set_penwidth(penwidth)
    node1.set_fillcolor('white')
    node1.set_color(aramco_color)
    image_path1 = get_image_path(species1)
    if os.path.exists(image_path1):
        node1.set_image(image_path1)
        node1.set_label("")

    index2 = species1_to_2[index1]
    if index2 >= 0:
        concentration2 = concentrations2[t2, index2] / max_concentration_total
        if concentration2 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration2) + max_node_pen_width, 3)
            nodewidths[1] = penwidth
            if node1.get_penwidth() > 0:
                node1.set_color('black')
            else:
                node1.set_color(rmg1_color)
                node1.set_penwidth(penwidth)

    index3 = species1_to_3[index1]
    if index3 >= 0:
        concentration3 = concentrations3[t3, index3] / max_concentration3
        if concentration3 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration3) + max_node_pen_width, 3)
            nodewidths[2] = penwidth
            if node1.get_penwidth() > 0:
                node1.set_color('black')
            else:
                node1.set_color(rmg7_color)
                node1.set_penwidth(penwidth)     

    if node1.get_color() == 'black':
        node1.set_penwidth(np.average(nodewidths))
    graph.add_node(node1)

# For Mechanism 2
for index2 in nodes2:
    if species2_to_1[index2] in nodes1:
        continue  # already took care of it above

    nodewidths = np.zeros(3)
    species2 = species_list2[index2]
    node2 = pydot.Node(name=str(species2))
    concentration2 = concentrations2[t2, index2] / max_concentration_total
    if concentration2 < concentration_tol:
        penwidth = 0.0
    else:
        penwidth = round(slope * np.log10(concentration2) + max_node_pen_width, 3)
        nodewidths[1] = penwidth
    node2.set_fillcolor('white')
    node2.set_color(rmg1_color)
    node2.set_penwidth(penwidth)
    # Try to use an image instead of the label
    image_path2 = get_image_path(species2)
    if os.path.exists(image_path2):
        node2.set_image(image_path2)
        node2.set_label("")

    index3 = species2_to_3[index2]
    if index3 >= 0:
        concentration3 = concentrations3[t3, index3] / max_concentration3
        if concentration3 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration3) + max_node_pen_width, 3)
            if node2.get_penwidth() > 0:
                node2.set_color('black')
            else:
                node2.set_color(rmg7_color)
                node2.set_penwidth(penwidth)
                nodewidths[2] = penwidth

    if node2.get_color() == 'black':
        node2.set_penwidth(np.average(nodewidths))

    graph.add_node(node2)

# For Mechanism 3
for index3 in nodes3:
    if species3_to_1[index3] in nodes1 or species3_to_2[index3] in nodes2:
        continue  # already covered above

    nodewidths = np.zeros(3)

    species3 = species_list3[index3]
    node3 = pydot.Node(name=str(species3))
    concentration3 = concentrations3[t3, index3] / max_concentration3
    if concentration3 < concentration_tol:
        penwidth = 0.0
    else:
        penwidth = round(slope * np.log10(concentration3) + max_node_pen_width, 3)
        nodewidths[2] = penwidth

    node3.set_fillcolor('white')
    node3.set_color(rmg7_color)
    node3.set_penwidth(penwidth)

    # Try to use an image instead of the label
    image_path3 = get_image_path(species3)
    if os.path.exists(image_path3):
        node3.set_image(image_path3)
        node3.set_label("")

    if node3.get_color() == 'black':
        # need to get concentrations for 2 and 1            
        node3.set_penwidth(np.average(nodewidths))

    graph.add_node(node3)



# ------------------------------- EDGES ------------------------------#
# Add an edge for each species-species rate
slope = -max_edge_pen_width / np.log10(species_rate_tol)

# Go through edges in Mechanism 1
for reactant_index1, product_index1 in edges1:
    if reactant_index1 in nodes1 and product_index1 in nodes1:
        reactant1 = species_list1[reactant_index1]
        product1 = species_list1[product_index1]
        label1 = label_flux_pair(reactant1, product1)
        
        edge1 = pydot.Edge(str(reactant1), str(product1), color=aramco_color)
        species_rate1 = species_rates1[t1, reactant_index1, product_index1] / max_species_rate_total
        if species_rate1 < 0:
            edge1.set_dir("back")
            species_rate1 = -species_rate1
        else:
            edge1.set_dir("forward")
        # Set the edge pen width
        if species_rate1 < species_rate_tol:
            penwidth = 0.0
            edge1.set_dir("none")
        else:
            penwidth = round(slope * np.log10(species_rate1) + max_edge_pen_width, 3)
        edge1.set_penwidth(penwidth)
        if label1 and label_automatically:
            edge1.set_decorate(True)
            edge1.set_label(label1)

        graph.add_edge(edge1)

        # add mech 2
        if species1_to_2[reactant_index1] >= 0 and species1_to_2[product_index1] >= 0:
            reactant_index2 = species1_to_2[reactant_index1]
            product_index2 = species1_to_2[product_index1]

            edge2 = pydot.Edge(str(reactant1), str(product1), color=rmg1_color)
            species_rate2 = species_rates2[t2, reactant_index2, product_index2] / max_species_rate_total
            if species_rate2 < 0:
                edge2.set_dir("back")
                species_rate2 = -species_rate2
            else:
                edge2.set_dir("forward")
            # Set the edge pen width
            if species_rate2 < species_rate_tol:
                penwidth = 0.0
                edge2.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate2) + max_edge_pen_width, 3)
            edge2.set_penwidth(penwidth)
            graph.add_edge(edge2)

        # add mech 3
        if species1_to_3[reactant_index1] >= 0 and species1_to_3[product_index1] >= 0:
            reactant_index3 = species1_to_3[reactant_index1]
            product_index3 = species1_to_3[product_index1]

            edge3 = pydot.Edge(str(reactant1), str(product1), color=rmg7_color)
            species_rate3 = species_rates3[t3, reactant_index3, product_index3] / max_species_rate_total
            if species_rate3 < 0:
                edge3.set_dir("back")
                species_rate3 = -species_rate3
            else:
                edge3.set_dir("forward")
            # Set the edge pen width
            if species_rate3 < species_rate_tol:
                penwidth = 0.0
                edge3.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate3) + max_edge_pen_width, 3)
            edge3.set_penwidth(penwidth)            
            graph.add_edge(edge3)

# Go through edges in Mechanism 2
for reactant_index2, product_index2 in edges2:
    # skip if this was already done in edges 1
    if [species2_to_1[reactant_index2], species2_to_1[product_index2]] in edges1 or \
        [species2_to_1[product_index2], species2_to_1[reactant_index2]] in edges1:
        continue

    if reactant_index2 in nodes2 and product_index2 in nodes2:
        # mech 2 says include this edge for all mechs
        if species2_to_1[reactant_index2] in nodes1:
            reactant2 = species_list1[species2_to_1[reactant_index2]]
        else:
            reactant2 = species_list2[reactant_index2]
        if species2_to_1[product_index2] in nodes1:
            product2 = species_list1[species2_to_1[product_index2]]
        else:
            product2 = species_list2[product_index2]
        edge2 = pydot.Edge(str(reactant2), str(product2), color=rmg1_color)
            
        species_rate2 = species_rates2[t2, reactant_index2, product_index2] / max_species_rate_total
        if species_rate2 < 0:
            edge2.set_dir("back")
            species_rate2 = -species_rate2
        else:
            edge2.set_dir("forward")
        # Set the edge pen width
        if species_rate2 < species_rate_tol:
            penwidth = 0.0
            edge2.set_dir("none")
        else:
            penwidth = round(slope * np.log10(species_rate2) + max_edge_pen_width, 3)



        edge2.set_penwidth(penwidth)
        graph.add_edge(edge2)

        # add mech 1
        if species2_to_1[reactant_index2] >= 0 and species2_to_1[product_index2] >= 0:
            reactant_index1 = species2_to_1[reactant_index2]
            product_index1 = species2_to_1[product_index2]

            edge1 = pydot.Edge(str(reactant2), str(product2), color=aramco_color)
            species_rate1 = species_rates1[t1, reactant_index1, product_index1] / max_species_rate_total
            if species_rate1 < 0:
                edge1.set_dir("back")
                species_rate1 = -species_rate1
            else:
                edge1.set_dir("forward")
            # Set the edge pen width
            if species_rate1 < species_rate_tol:
                penwidth = 0.0
                edge1.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate1) + max_edge_pen_width, 3)
            edge1.set_penwidth(penwidth)
            label2 = label_flux_pair(reactant2, product2)  # <--------------------
            if label2 and label_automatically:
                edge1.set_decorate(True)
                edge1.set_label(label2)
            graph.add_edge(edge1)

        # add mech 3
        if species2_to_3[reactant_index2] >= 0 and species2_to_3[product_index2] >= 0:
            reactant_index3 = species2_to_3[reactant_index2]
            product_index3 = species2_to_3[product_index2]

            edge3 = pydot.Edge(str(reactant2), str(product2), color=rmg7_color)
            species_rate3 = species_rates3[t3, reactant_index3, product_index3] / max_species_rate_total
            if species_rate3 < 0:
                edge3.set_dir("back")
                species_rate3 = -species_rate3
            else:
                edge3.set_dir("forward")
            # Set the edge pen width
            if species_rate3 < species_rate_tol:
                penwidth = 0.0
                edge3.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate3) + max_edge_pen_width, 3)
            edge3.set_penwidth(penwidth)
            graph.add_edge(edge3)


# Go through edges in Mechanism 3
for reactant_index3, product_index3 in edges3:
    # skip if this was already done in edges 1 or edges 2
    if [species3_to_1[reactant_index3], species3_to_1[product_index3]] in edges1 or \
        [species3_to_1[product_index3], species3_to_1[reactant_index3]] in edges1:
        continue
    if [species3_to_2[reactant_index3], species3_to_2[product_index3]] in edges2 or \
        [species3_to_2[product_index3], species3_to_2[reactant_index3]] in edges2:
        continue

    if reactant_index3 in nodes3 and product_index3 in nodes3:
        if species3_to_1[reactant_index3] in nodes1:
            reactant3 = species_list1[species3_to_1[reactant_index3]]
        elif species3_to_2[reactant_index3] in nodes2:
            reactant3 = species_list2[species3_to_2[reactant_index3]]
        else:
            reactant3 = species_list3[reactant_index3]

        if species3_to_1[product_index3] in nodes1:
            product3 = species_list1[species3_to_1[product_index3]]
        elif species3_to_2[product_index3] in nodes2:
            product3 = species_list2[species3_to_2[product_index3]]
        else:
            product3 = species_list3[product_index3]
        edge3 = pydot.Edge(str(reactant3), str(product3), color=rmg7_color)
        species_rate3 = species_rates3[t3, reactant_index3, product_index3] / max_species_rate_total
        if species_rate3 < 0:
            edge3.set_dir("back")
            species_rate3 = -species_rate3
        else:
            edge3.set_dir("forward")
        # Set the edge pen width
        if species_rate3 < species_rate_tol:
            penwidth = 0.0
            edge3.set_dir("none")
        else:
            penwidth = round(slope * np.log10(species_rate3) + max_edge_pen_width, 3)

        edge3.set_penwidth(penwidth)
        graph.add_edge(edge3)

        # add mech 1
        if species3_to_1[reactant_index3] >= 0 and species3_to_1[product_index3] >= 0:
            reactant_index1 = species3_to_1[reactant_index3]
            product_index1 = species3_to_1[product_index3]

            edge1 = pydot.Edge(str(reactant3), str(product3), color=aramco_color)
            species_rate1 = species_rates1[t1, reactant_index1, product_index1] / max_species_rate_total
            if species_rate1 < 0:
                edge1.set_dir("back")
                species_rate1 = -species_rate1
            else:
                edge1.set_dir("forward")
            # Set the edge pen width
            if species_rate1 < species_rate_tol:
                penwidth = 0.0
                edge1.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate1) + max_edge_pen_width, 3)
            edge1.set_penwidth(penwidth)
            
            label3 = label_flux_pair(reactant3, product3)  # <--------------------
            if label3 and label_automatically:
                edge1.set_decorate(True)
                edge1.set_label(label3)
            graph.add_edge(edge1)


        # add mech 2
        if species3_to_2[reactant_index3] >= 0 and species3_to_2[product_index3] >= 0:
            reactant_index2 = species3_to_2[reactant_index3]
            product_index2 = species3_to_2[product_index3]

            edge2 = pydot.Edge(str(reactant3), str(product3), color=rmg1_color)
            
            species_rate2 = species_rates2[t2, reactant_index2, product_index2] / max_species_rate_total
            if species_rate2 < 0:
                edge2.set_dir("back")
                species_rate2 = -species_rate2
            else:
                edge2.set_dir("forward")
            # Set the edge pen width
            if species_rate2 < species_rate_tol:
                penwidth = 0.0
                edge2.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate2) + max_edge_pen_width, 3)
            edge2.set_penwidth(penwidth)
            graph.add_edge(edge2)


# General purpose graph settings
graph.set_nodesep(0.11)
graph.set_ranksep(0.35)
graph.set_rankdir('LR')

# Add Legend
graph.add_node(pydot.Node(mech_1_label + f'\nt={times1[t1]:.4e}', label=mech_1_label + f'\nt={times1[t1]:.4e}', color=aramco_color, shape='box', penwidth=max_node_pen_width))
graph.add_node(pydot.Node(mech_3_label + f'\nt={times3[t3]:.4e}', label=mech_3_label + f'\nt={times3[t3]:.4e}', color=rmg7_color, shape='box', penwidth=max_node_pen_width))
graph.add_node(pydot.Node(mech_2_label + f'\nt={times2[t2]:.4e}', label=mech_2_label + f'\nt={times2[t2]:.4e}', color=rmg1_color, shape='box', penwidth=max_node_pen_width))


# write in multiple formats
graph.write_dot(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.dot')) # Yes this is supposed to be an index, not an actual time
graph.write_png(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.png'))
graph.write_pdf(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.pdf'))
graph.write_pdf(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.svg'))




In [ ]:
# Graph labels added manually to .svg file afterwards (using Adobe InDesign)